In [1]:
ds = SurfaceDataset(sequences)
len(ds)

In [2]:
# from google.colab import drive
from pathlib import Path
# drive.mount('/content/gdrive', force_remount=True)
%cd "/content/drive/Othercomputers/my_mac/Coding_stuff/Projects/Surface_Predictor"

In [3]:
creds = '...'
cred_path = Path('~/.kaggle/kaggle.json').expanduser()
if not cred_path.exists():
  print('entered')
  cred_path.parent.mkdir(exist_ok=True)
  cred_path.write_text(creds)
  cred_path.chmod(0o600)

In [4]:
# !kaggle competitions download -c career-con-2019 -p ./data/
# !unzip ./data/X_test.csv.zip -d ./data
# !unzip ./data/X_train.csv.zip -d ./data
# !rm ./data/X_train.csv.zip ./data/X_test.csv.zip

In [5]:
%pip install -Uqq pytorch-lightning torchmetrics wandb

In [6]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import rc
import seaborn as sns
import wandb
from tqdm.auto import tqdm 

import torch
import torch.autograd as autograd
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

import pytorch_lightning as pl
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping
from pytorch_lightning.callbacks.progress import TQDMProgressBar
from pytorch_lightning.loggers import WandbLogger
import torchmetrics

from pylab import rcParams
from matplotlib.ticker import MaxNLocator
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

In [7]:
%matplotlib inline
%config InlinBackend.figure_format = 'retina'
sns.set_style(style= 'whitegrid')
rcParams['figure.figsize'] = 14,9

In [8]:
pl.seed_everything(42)

42

In [9]:
%matplotlib inline
%config InlinBackend.figure_format = 'retina'
sns.set_style(style= 'whitegrid')
rcParams['figure.figsize'] = 14,9

In [10]:
pl.seed_everything(42)

42

In [11]:
!wandb login

In [12]:
!pwd

In [13]:
x_train = pd.read_csv('./data/raw/X_train.csv', low_memory=False)
y_train = pd.read_csv('./data/raw/y_train.csv', low_memory=False)
x_train

          row_id  series_id  ...  linear_acceleration_Y  linear_acceleration_Z
0            0_0          0  ...                 2.1030                -9.7532
1            0_1          0  ...                 1.5064                -9.4128
2            0_2          0  ...                 1.5922                -8.7267
3            0_3          0  ...                 1.0993               -10.0960
4            0_4          0  ...                 1.4689               -10.4410
...          ...        ...  ...                    ...                    ...
487675  3809_123       3809  ...                 2.0115                -9.0063
487676  3809_124       3809  ...                 3.0696                -8.1257
487677  3809_125       3809  ...                 4.2622                -8.1443
487678  3809_126       3809  ...                 4.7130                -9.4435
487679  3809_127       3809  ...                 4.2751               -10.4980

[487680 rows x 13 columns]

In [14]:
x_train.info()

In [15]:
y_train

      series_id  group_id        surface
0             0        13  fine_concrete
1             1        31       concrete
2             2        20       concrete
3             3        31       concrete
4             4        22     soft_tiles
...         ...       ...            ...
3805       3805        55          tiled
3806       3806        67           wood
3807       3807        48  fine_concrete
3808       3808        54          tiled
3809       3809        56       soft_pvc

[3810 rows x 3 columns]

In [16]:
y_train.surface.value_counts().plot(kind='bar')
plt.xticks(rotation=45)
plt.title('Distribution of target (surface) ')
plt.show()

In [17]:
y_train.surface.value_counts().plot(kind='bar')
plt.xticks(rotation=45)
plt.title('Distribution of target (surface) ')
plt.show()

In [18]:
label_encoder =  LabelEncoder()
encoded_labels = label_encoder.fit_transform(y_train.surface)

In [19]:
label_encoder.classes_

array(['carpet', 'concrete', 'fine_concrete', 'hard_tiles',
       'hard_tiles_large_space', 'soft_pvc', 'soft_tiles', 'tiled',
       'wood'], dtype=object)

In [20]:
encoded_labels[:5], list(y_train.surface[:5])

(array([2, 1, 1, 1, 6]),
 ['fine_concrete', 'concrete', 'concrete', 'concrete', 'soft_tiles'])

In [21]:
y_train['label'] = encoded_labels
y_train

      series_id  group_id        surface  label
0             0        13  fine_concrete      2
1             1        31       concrete      1
2             2        20       concrete      1
3             3        31       concrete      1
4             4        22     soft_tiles      6
...         ...       ...            ...    ...
3805       3805        55          tiled      7
3806       3806        67           wood      8
3807       3807        48  fine_concrete      2
3808       3808        54          tiled      7
3809       3809        56       soft_pvc      5

[3810 rows x 4 columns]

In [22]:
x_train

          row_id  series_id  ...  linear_acceleration_Y  linear_acceleration_Z
0            0_0          0  ...                 2.1030                -9.7532
1            0_1          0  ...                 1.5064                -9.4128
2            0_2          0  ...                 1.5922                -8.7267
3            0_3          0  ...                 1.0993               -10.0960
4            0_4          0  ...                 1.4689               -10.4410
...          ...        ...  ...                    ...                    ...
487675  3809_123       3809  ...                 2.0115                -9.0063
487676  3809_124       3809  ...                 3.0696                -8.1257
487677  3809_125       3809  ...                 4.2622                -8.1443
487678  3809_126       3809  ...                 4.7130                -9.4435
487679  3809_127       3809  ...                 4.2751               -10.4980

[487680 rows x 13 columns]

In [23]:
#we dont need the first 3 cols
feature_columns = x_train.columns.to_list()[3:]
feature_columns

['orientation_X',
 'orientation_Y',
 'orientation_Z',
 'orientation_W',
 'angular_velocity_X',
 'angular_velocity_Y',
 'angular_velocity_Z',
 'linear_acceleration_X',
 'linear_acceleration_Y',
 'linear_acceleration_Z']

In [24]:
#it looks like the sequences have been grouped for us
x_train.series_id.value_counts()

0       128
2544    128
2532    128
2533    128
2534    128
       ... 
1274    128
1275    128
1276    128
1277    128
3809    128
Name: series_id, Length: 3810, dtype: int64

In [25]:
sequences = []
for series_id, group in x_train.groupby('series_id'):
  sequence_features = group[feature_columns]
  label = y_train[y_train.series_id == series_id].iloc[0].label
  sequences.append((sequence_features, label))

In [26]:
sequences[0]

(     orientation_X  orientation_Y  ...  linear_acceleration_Y  linear_acceleration_Z
 0         -0.75853       -0.63435  ...                2.10300                -9.7532
 1         -0.75853       -0.63434  ...                1.50640                -9.4128
 2         -0.75853       -0.63435  ...                1.59220                -8.7267
 3         -0.75852       -0.63436  ...                1.09930               -10.0960
 4         -0.75852       -0.63435  ...                1.46890               -10.4410
 ..             ...            ...  ...                    ...                    ...
 123       -0.75943       -0.63316  ...                1.79890                -6.8687
 124       -0.75945       -0.63313  ...                2.35620                -6.8926
 125       -0.75949       -0.63309  ...                0.97328                -9.9795
 126       -0.75950       -0.63307  ...                0.34931               -11.9770
 127       -0.75953       -0.63306  ...               

In [27]:
from torch.utils.data import TensorDataset, WeightedRandomSampler

In [28]:
all_label_ids = torch.tensor([seq[1] for seq in sequences], dtype=torch.long)
all_label_ids

tensor([2, 1, 1,  ..., 2, 7, 5])

In [29]:
labels_unique, counts = np.unique(all_label_ids, return_counts=True)
labels_unique, counts

(array([0, 1, 2, 3, 4, 5, 6, 7, 8]),
 array([189, 779, 363,  21, 308, 732, 297, 514, 607]))

In [30]:

class SurfaceDataset(Dataset):

  def __init__(self, sequences) -> None:
      super().__init__()
      self.sequences = sequences

  def __len__(self):
    return len(self.sequences)

  def __getitem__(self, index):
    sequence, label = self.sequences[index]
    return dict(
      sequence=torch.Tensor(sequence.to_numpy()),
      label=torch.tensor(label).long()
    )

In [31]:

class SurfaceDataset(Dataset):

  def __init__(self, sequences) -> None:
      super().__init__()
      self.sequences = sequences

  def __len__(self):
    return len(self.sequences)

  def __getitem__(self, index):
    sequence, label = self.sequences[index]
    return dict(
      sequence=torch.Tensor(sequence.to_numpy()),
      label=torch.tensor(label).long()
    )

In [32]:
ds = SurfaceDataset(sequences)
len(ds)

3810

In [33]:
ds = SurfaceDataset(sequences)
len(ds)
ds[0]

{'label': tensor(2),
 'sequence': tensor([[ -0.7585,  -0.6344,  -0.1049,  ...,  -0.7486,   2.1030,  -9.7532],
         [ -0.7585,  -0.6343,  -0.1049,  ...,   0.3399,   1.5064,  -9.4128],
         [ -0.7585,  -0.6344,  -0.1049,  ...,  -0.2643,   1.5922,  -8.7267],
         ...,
         [ -0.7595,  -0.6331,  -0.1047,  ...,  -0.9819,   0.9733,  -9.9795],
         [ -0.7595,  -0.6331,  -0.1046,  ...,  -0.7259,   0.3493, -11.9770],
         [ -0.7595,  -0.6331,  -0.1046,  ...,  -0.5343,   0.8035, -10.6970]])}

In [34]:
ds = SurfaceDataset(sequences)
len(ds)
[d['labelds'] for d in ds]

In [35]:
ds = SurfaceDataset(sequences)
len(ds)
[d['label'] for d in ds]

[tensor(2),
 tensor(1),
 tensor(1),
 tensor(1),
 tensor(6),
 tensor(7),
 tensor(5),
 tensor(1),
 tensor(4),
 tensor(7),
 tensor(5),
 tensor(7),
 tensor(0),
 tensor(0),
 tensor(1),
 tensor(0),
 tensor(0),
 tensor(6),
 tensor(7),
 tensor(6),
 tensor(7),
 tensor(4),
 tensor(6),
 tensor(7),
 tensor(6),
 tensor(6),
 tensor(2),
 tensor(3),
 tensor(7),
 tensor(4),
 tensor(6),
 tensor(6),
 tensor(2),
 tensor(1),
 tensor(1),
 tensor(1),
 tensor(7),
 tensor(0),
 tensor(5),
 tensor(8),
 tensor(2),
 tensor(8),
 tensor(2),
 tensor(8),
 tensor(6),
 tensor(3),
 tensor(5),
 tensor(2),
 tensor(1),
 tensor(5),
 tensor(1),
 tensor(6),
 tensor(6),
 tensor(1),
 tensor(0),
 tensor(6),
 tensor(2),
 tensor(2),
 tensor(0),
 tensor(6),
 tensor(1),
 tensor(7),
 tensor(5),
 tensor(4),
 tensor(5),
 tensor(1),
 tensor(7),
 tensor(7),
 tensor(1),
 tensor(5),
 tensor(1),
 tensor(1),
 tensor(5),
 tensor(5),
 tensor(8),
 tensor(2),
 tensor(6),
 tensor(2),
 tensor(5),
 tensor(2),
 tensor(6),
 tensor(7),
 tensor(5),
 ten

In [36]:
ds = SurfaceDataset(sequences)
len(ds)
lab_ids = [d['label'] for d in ds]
len(lab_ids)

3810

In [37]:
labels_unique, counts = np.unique(lab_ids, return_counts=True)
labels_unique, counts

(array([0, 1, 2, 3, 4, 5, 6, 7, 8]),
 array([189, 779, 363,  21, 308, 732, 297, 514, 607]))

In [38]:
class_weights = [sum(counts)/c for c in counts]
print(class_weights)

In [39]:
sample_weights = [class_weights[e] for e in lab_ids ]
weights

In [40]:
sample_weights = [class_weights[e] for e in lab_ids ]
sample_weights

[10.49586776859504,
 4.890885750962773,
 4.890885750962773,
 4.890885750962773,
 12.828282828282829,
 7.412451361867705,
 5.204918032786885,
 4.890885750962773,
 12.37012987012987,
 7.412451361867705,
 5.204918032786885,
 7.412451361867705,
 20.158730158730158,
 20.158730158730158,
 4.890885750962773,
 20.158730158730158,
 20.158730158730158,
 12.828282828282829,
 7.412451361867705,
 12.828282828282829,
 7.412451361867705,
 12.37012987012987,
 12.828282828282829,
 7.412451361867705,
 12.828282828282829,
 12.828282828282829,
 10.49586776859504,
 181.42857142857142,
 7.412451361867705,
 12.37012987012987,
 12.828282828282829,
 12.828282828282829,
 10.49586776859504,
 4.890885750962773,
 4.890885750962773,
 4.890885750962773,
 7.412451361867705,
 20.158730158730158,
 5.204918032786885,
 6.276771004942339,
 10.49586776859504,
 6.276771004942339,
 10.49586776859504,
 6.276771004942339,
 12.828282828282829,
 181.42857142857142,
 5.204918032786885,
 10.49586776859504,
 4.890885750962773,
 5.2

In [41]:
type(sampler)

In [42]:
sampler = WeightedRandomSampler(sample_weights,  num_samples=len(sample_weights), replacement=True)

In [43]:
type(sampler)

torch.utils.data.sampler.WeightedRandomSampler

In [44]:
sampler

In [45]:
def get_sampler(dataset):
    lab_ids = [d['label'] for d in dataset]
    labels_unique, counts = np.unique(lab_ids, return_counts=True)
    class_weights = [sum(counts)/c for c in counts]
    sample_weights = [class_weights[e] for e in lab_ids ]
    sampler = WeightedRandomSampler(sample_weights,  num_samples=len(sample_weights), replacement=True)
    return sampler

In [46]:
train_sequences, val_sequences = train_test_split(sequences, test_size=0.2)
len(train_sequences), len(val_sequences)

(3048, 762)

In [47]:
train_dataset = SurfaceDataset(train_sequences)
len(train_dataset)

3048

In [48]:
lab_ids = [d['label'] for d in dataset]
labels_unique, counts = np.unique(lab_ids, return_counts=True)

In [49]:
lab_ids = [item['label'] for item in train_dataset]
labels_unique, counts = np.unique(lab_ids, return_counts=True)
labels_unique, counts

(array([0, 1, 2, 3, 4, 5, 6, 7, 8]),
 array([142, 607, 298,  16, 245, 591, 234, 425, 490]))

In [50]:
#lets plot them
sns.countplot(x=labels_unique, y=counts)

In [51]:
#lets plot them
sns.countplot(x=labels_unique, data=counts)

In [52]:
#lets plot them
sns.countplot(x=labels_unique, data=lab_ids)

In [53]:
#lets plot them
sns.barplot(x=labels_unique,y=counts)

In [54]:
#lets plot them
sns.barplot(x=label_encoder.classes_,y=counts)

In [55]:
#lets plot them
sns.barplot(x=label_encoder.classes_,y=counts)
plt.xticks(rotation=30)

(array([0, 1, 2, 3, 4, 5, 6, 7, 8]),
 <a list of 9 Text major ticklabel objects>)

In [56]:
#lets plot them
ax = sns.barplot(data=counts)
ax.bar_label(ax.containers[0])
# sns.barplot(x=label_encoder.classes_,y=counts)
# plt.xticks(rotation=30)

In [57]:
#lets plot them
# ax = sns.barplot(data=counts)
# ax.bar_label(ax.containers[0])
ax = sns.barplot(x=label_encoder.classes_,y=counts)
ax.bar_label(ax.containers[0])
plt.xticks(rotation=30)

In [58]:
#lets plot them
# ax = sns.barplot(data=counts)
# ax.bar_label(ax.containers[0])
ax = sns.barplot(x=label_encoder.classes_,y=counts)

ax.xticks(rotation=30)

In [59]:
#lets plot them
# ax = sns.barplot(data=counts)
# ax.bar_label(ax.containers[0])
ax = sns.barplot(x=label_encoder.classes_,y=counts)

ax.set_xticks(rotation=30)

In [60]:
#lets plot them
# ax = sns.barplot(data=counts)
# ax.bar_label(ax.containers[0])
ax = sns.barplot(x=label_encoder.classes_,y=counts)

ax.set_xticks(label_encoder.classes_, rotation=30)

In [61]:
#lets plot them
# ax = sns.barplot(data=counts)
# ax.bar_label(ax.containers[0])
ax = sns.barplot(x=label_encoder.classes_,y=counts)

ax.set_xticklabels(ax.get_xticklabels(),rotation=30)

[Text(0, 0, 'carpet'),
 Text(0, 0, 'concrete'),
 Text(0, 0, 'fine_concrete'),
 Text(0, 0, 'hard_tiles'),
 Text(0, 0, 'hard_tiles_large_space'),
 Text(0, 0, 'soft_pvc'),
 Text(0, 0, 'soft_tiles'),
 Text(0, 0, 'tiled'),
 Text(0, 0, 'wood')]

In [62]:
#lets plot them
# ax = sns.barplot(data=counts)
# ax.bar_label(ax.containers[0])
ax = sns.barplot(x=label_encoder.classes_,y=counts)

ax.set_xticklabels(ax.get_xticklabels(),rotation=30);

In [63]:
#lets plot them
# ax = sns.barplot(data=counts)
# ax.bar_label(ax.containers[0])
ax = sns.barplot(x=label_encoder.classes_,y=counts)

ax.set_xticklabels(ax.get_xticklabels(),rotation=30)
i=0
for p in ax.patches:
    height = p.get_height()
    ax.text(p.get_x()+p.get_width()/2., height + 0.1,
        counts[i],ha="center")
    i += 1
;

''

In [64]:
#lets plot them
# ax = sns.barplot(data=counts)
# ax.bar_label(ax.containers[0])
ax = sns.barplot(x=label_encoder.classes_,y=counts)

ax.set_xticklabels(ax.get_xticklabels(),rotation=30)
i=0
for p in ax.patches:
    height = p.get_height()
    ax.text(p.get_x()+p.get_width()/2., height + 0.1,
        counts[i],ha="center", size=18)
    i += 1
;

''

In [65]:
class_weights = [sum(counts)/c for c in counts]
class_weight

In [66]:
class_weights = [sum(counts)/c for c in counts]
class_weights

[21.464788732394368,
 5.021416803953872,
 10.228187919463087,
 190.5,
 12.440816326530612,
 5.157360406091371,
 13.025641025641026,
 7.171764705882353,
 6.220408163265306]

In [67]:
sample_weights = [class_weights[e] for e in lab_ids ]
sample_weights

[12.440816326530612,
 12.440816326530612,
 5.157360406091371,
 12.440816326530612,
 5.157360406091371,
 12.440816326530612,
 5.021416803953872,
 13.025641025641026,
 7.171764705882353,
 12.440816326530612,
 5.157360406091371,
 10.228187919463087,
 5.157360406091371,
 5.157360406091371,
 5.157360406091371,
 7.171764705882353,
 12.440816326530612,
 6.220408163265306,
 5.157360406091371,
 21.464788732394368,
 13.025641025641026,
 10.228187919463087,
 5.021416803953872,
 7.171764705882353,
 10.228187919463087,
 7.171764705882353,
 5.021416803953872,
 5.157360406091371,
 12.440816326530612,
 6.220408163265306,
 5.021416803953872,
 13.025641025641026,
 5.021416803953872,
 5.021416803953872,
 5.157360406091371,
 13.025641025641026,
 7.171764705882353,
 10.228187919463087,
 13.025641025641026,
 5.021416803953872,
 12.440816326530612,
 6.220408163265306,
 6.220408163265306,
 12.440816326530612,
 10.228187919463087,
 7.171764705882353,
 6.220408163265306,
 21.464788732394368,
 7.171764705882353,

In [68]:
sampler = WeightedRandomSampler(sample_weights,  num_samples=len(sample_weights), replacement=True)
sample

In [69]:
sampler = WeightedRandomSampler(sample_weights,  num_samples=len(sample_weights), replacement=True)
sampler

In [70]:
# lets check what this will yield when loading the dataset with a DataLoader
train_dl = DataLoader(train_dataset, sampler=sampler, batch_size=100)
batches = []
for idx, batch in train_dl:
    x,y = batch['sequence'], batch['label']
    sns.countplot(y)
    break

In [71]:
# lets check what this will yield when loading the dataset with a DataLoader
train_dl = DataLoader(train_dataset, sampler=sampler, batch_size=100)
batches = []
for idx, batch in train_dl:
    print(batch)
    x,y = batch['sequence'], batch['label']
    sns.countplot(y)
    break

In [72]:
# lets check what this will yield when loading the dataset with a DataLoader
train_dl = DataLoader(train_dataset, sampler=sampler, batch_size=100)
batches = []
for idx, batch in enumerate(train_dl):
    print(batch)
    x,y = batch['sequence'], batch['label']
    sns.countplot(y)
    break

In [73]:
# lets check what this will yield when loading the dataset with a DataLoader
train_dl = DataLoader(train_dataset, sampler=sampler, batch_size=100)
batches = []
for idx, batch in enumerate(train_dl):
    x,y = batch['sequence'], batch['label']
    sns.countplot(y)
    break

In [74]:
# lets check what this will yield when loading the dataset with a DataLoader
train_dl = DataLoader(train_dataset, sampler=sampler, batch_size=100)
batches = []
for idx, batch in enumerate(train_dl):
    x,y = batch['sequence'], batch['label']
    _, c = np.unique(y, return_counts=True)
    ax = sns.barplot(x=label_encoder.classes_,y=c)

    ax.set_xticklabels(ax.get_xticklabels(),rotation=30)
    i=0
    for p in ax.patches:
        height = p.get_height()
        ax.text(p.get_x()+p.get_width()/2., height + 0.1,
            c[i],ha="center", size=18)
        i += 1
    break

In [75]:
# lets check what this will yield when loading the dataset with a DataLoader
train_dl = DataLoader(train_dataset, sampler=sampler, batch_size=100)
batches = []
for idx, batch in enumerate(train_dl):
    x,y = batch['sequence'], batch['label']
    _, c = np.unique(y, return_counts=True)
    ax = sns.barplot(x=label_encoder.classes_,y=c)

    ax.set_xticklabels(ax.get_xticklabels(),rotation=30)
    i=0
    for p in ax.patches:
        height = p.get_height()
        ax.text(p.get_x()+p.get_width()/2., height + 0.1,
            c[i],ha="center", size=18)
        i += 1
    break

In [76]:
# see all train data
batches = []
for idx, batch in enumerate(train_dl):
    x,y = batch['sequence'], batch['label']
    batches.append(y.numpy())

In [77]:
_, c = np.unique(np.concatenate(batches), return_counts=True)
ax = sns.barplot(x=label_encoder.classes_,y=c)

ax.set_xticklabels(ax.get_xticklabels(),rotation=30)
i=0
for p in ax.patches:
    height = p.get_height()
    ax.text(p.get_x()+p.get_width()/2., height + 0.1,
        c[i],ha="center", size=18)
    i += 1

In [78]:
class SurfaceDatamodule(pl.LightningDataModule):

  def __init__(self, train_sequences, val_sequences, batch_size, balance=True):
    super().__init__()
    self.train_sequences = train_sequences
    self.val_sequences = val_sequences
    self.batch_size = batch_size
    self.balance = balance
    self.sampler = None

  def setup(self, stage=None):
    self.train_dataset = SurfaceDataset(self.train_sequences)
    self.val_dataset = SurfaceDataset(self.val_sequences)
    if self.balance:
      self.sampler = get_sampler(self.train_dataset)

  def train_dataloader(self):
    if self.balance:
      return DataLoader(self.train_dataset, 
                        batch_size=self.batch_size,
                        sampler=self.sampler,
                        num_workers=3)
    return DataLoader(self.train_dataset, 
                  batch_size=self.batch_size,
                  num_workers=3)
      
  def val_dataloader(self):
      return DataLoader(self.val_dataset,
                        batch_size=self.batch_size,
                        num_workers=2)

In [79]:

class SurfaceModel(nn.Module):

  def __init__(self, n_features, n_classes, n_hidden,dropout, n_layers=3):
    super().__init__()
    print(dropout)
    self.n_fearues = n_features
    self.n_classes = n_classes
    self.n_hidden = n_hidden
    self.n_layers = n_layers
    self.dropout = dropout

    self.lstm = nn.LSTM(
        input_size =self.n_fearues,
        hidden_size = self.n_hidden,
        num_layers = self.n_layers,
        dropout = self.dropout,
        batch_first = True
    )
    self.cassifier = nn.Linear(self.n_hidden, self.n_classes)

  def forward(self, x):
        self.lstm.flatten_parameters()
        _, (hidden,_) = self.lstm(x)
        out = hidden[-1]
        return self.cassifier(out)

In [80]:
class SurfacePredictor(pl.LightningModule):
  def __init__(self, n_features, n_classes, n_hidden, seq_length, dropout, n_layers=3, lr=0.0001):
    super().__init__()
    self.lr = lr 
    self.seq_length =seq_length
    self.n_features = n_features

    self.model = SurfaceModel(n_features, n_classes, n_hidden, n_layers=n_layers,dropout=dropout)
    self.criterion = nn.CrossEntropyLoss()
    self.train_ac = torchmetrics.Accuracy()
    self.val_ac = torchmetrics.Accuracy()
    self.save_hyperparameters()


  def forward(self, x, labels=None):
    output = self.model(x)

    loss = 0
    if labels is not None:
      loss = self.criterion(output, labels)
      return loss, output

    return output


  def training_step(self, batch, batch_idx):
    sequences = batch['sequence']
    labels = batch['label']
    loss, outputs = self(sequences, labels)
    predictions = torch.argmax(outputs, dim=1)
    step_ac = self.train_ac(predictions, labels)

    self.log('train_loss', loss, prog_bar=True, logger=True)
    self.log('train_accuracy', step_ac, prog_bar=True, logger=True)

    return dict(loss=loss, accuracy=step_ac)

  def validation_step(self, batch, batch_idx):
    sequences = batch['sequence']
    labels = batch['label']
    loss, outputs = self(sequences, labels)
    predictions = torch.argmax(outputs, dim=1)
    step_ac = self.train_ac(predictions, labels)

    self.log('val_loss', loss, prog_bar=True, logger=True)
    self.log('val_accuracy', step_ac, prog_bar=True, logger=True)

    return dict(loss=loss, accuracy=step_ac, loggits=outputs)


  def validation_epoch_end(self, validation_step_outputs):
    dummyImput = torch.zeros(
        (1, self.seq_length, self.n_features), device=self.device)
    model_filename = f"model_{str(self.global_step).zfill(5)}.onnx"
    torch.onnx.export(self, dummyImput, model_filename, opset_version=11)
    # wandb.save(model_filename,)
    outputs = [out['loggits'] for out in validation_step_outputs]
    flattened_outputs = torch.flatten(
          torch.cat(outputs))
    self.logger.experiment.log(
    {'valid/logits': wandb.Histogram(flattened_outputs.to('cpu')),
      'epoch': self.current_epoch}
    )


  def configure_optimizers(self):
    return torch.optim.AdamW(self.parameters(), lr=self.lr)
 

In [81]:
def train(params, run_name):
    # setup data
    data_module =SurfaceDatamodule(train_sequences, val_sequences, batch_size=params['batch_size'])

    model = SurfacePredictor(n_features= params['n_features'], 
                             n_classes= params['n_classes'],
                             n_hidden= params['n_hidden'], 
                             seq_length = params['seq_length'],
                             dropout = params['dropout'], 
                             lr = params['lr'],
                             n_layers= params['n_layers'])
    # set up W&B logger
    checkpoint_callback = ModelCheckpoint(
    dirpath='checkpoints',
    filename='best_checkpoint',
    save_top_k=1,
    verbose=True,
    monitor='val_accuracy',
    mode='max'
)
    early_stopping_callback = EarlyStopping(monitor='val_accuracy', patience=10, verbose=True)
    progress_bar = TQDMProgressBar(refresh_rate=30)

    callbacks = [early_stopping_callback, checkpoint_callback, progress_bar]
    
    logger = WandbLogger(name=run_name, save_dir='logs', project='surface_pred')
    progress_bar = TQDMProgressBar(refresh_rate=30,)

    callbacks = [progress_bar]

    # setup Trainer
    trainer = pl.Trainer(gpus = 1,
                        max_epochs = 250,
                        logger= logger,
                        callbacks = callbacks,
                        log_every_n_steps=40,
                        precision=16)

    # train
    trainer.fit(model, datamodule=data_module)
    wandb.finish()
    return trainer, data_module

In [82]:
params ={
    'n_features': len(feature_columns),
    "n_classes": len(label_encoder.classes_),
    "seq_length":128,
    "dropout": 0.75,
    "n_hidden": 256,
    "batch_size": 64,
    "lr": 0.0001,
    "n_layers" : 3   
    }
trainer, data_module = train(params, 'run_41_yt-bal-data')

Validation sanity check: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

...

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]